# Next steps 

Add description

In [19]:
# Import statements
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

# Import the files we need 

In [20]:
VVR_measurements_clean = pd.read_csv('/Users/dionnespaltman/Desktop/V4/VVR_measurements/VVR_measurements_clean.csv', sep=',')
VVR_measurements_total = pd.read_csv('/Users/dionnespaltman/Desktop/V4/VVR_measurements/VVR_measurements.csv', sep=',')

if 'Unnamed: 0' in VVR_measurements_clean.columns:
    VVR_measurements_clean.drop(columns=['Unnamed: 0'], inplace=True)

if 'Unnamed: 0' in VVR_measurements_total.columns:
    VVR_measurements_total.drop(columns=['Unnamed: 0'], inplace=True)

display(VVR_measurements_clean)
display(VVR_measurements_total)

,ID,Sum_12,Sum_4567,VVR_1,VVR_2,Sum_456,VVR_group,Condition,Date,Gender
0,5,22.0,34.0,12.0,10.0,26.0,0,1,2019-10-21,2
1,6,16.0,32.0,8.0,8.0,24.0,0,2,2019-10-25,1
2,7,23.0,77.0,11.0,12.0,65.0,1,2,2019-10-28,2
3,8,17.0,32.0,9.0,8.0,24.0,0,1,2019-10-30,1
4,9,16.0,32.0,8.0,8.0,24.0,0,1,2019-10-31,1
...,...,...,...,...,...,...,...,...,...,...
315,328,20.0,35.0,10.0,10.0,27.0,0,3,2022-11-23,2
316,329,18.0,35.0,10.0,8.0,27.0,0,3,2022-11-28,2
317,330,23.0,35.0,11.0,12.0,26.0,0,3,2022-11-30,2
318,331,28.0,38.0,14.0,14.0,30.0,0,3,2022-11-30,2


,ID,Stage,Gender,Age,Date,Location,Condition,Weight,Length,Faintness,Dizziness,Weakness,Lightheadedness,Fear,Tension,Stress,Nervousness,Physical_sum,Psychological_sum
0,5,1.0,2,33.0,2019-10-21,0,1,74.0,171.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,4.0,8.0
1,5,2.0,2,33.0,2019-10-21,0,1,74.0,171.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,4.0,6.0
2,5,3.0,2,33.0,2019-10-21,0,1,74.0,171.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,4.0,6.0
3,5,4.0,2,33.0,2019-10-21,0,1,74.0,171.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,5.0
4,5,5.0,2,33.0,2019-10-21,0,1,74.0,171.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,332,2.0,2,21.0,2022-12-19,3,2,66.0,173.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,4.0,4.0,12.0
2045,332,4.0,2,21.0,2022-12-19,3,2,66.0,173.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,5.0,4.0,14.0
2046,332,5.0,2,21.0,2022-12-19,3,2,66.0,173.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,4.0,4.0,12.0
2047,332,6.0,2,21.0,2022-12-19,3,2,66.0,173.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,3.0,4.0,9.0


Loading the features we extracted from TS fresh. Here we only have data from 104 people. 

In [77]:
extracted_features_standardized = pd.read_csv('/Users/dionnespaltman/Desktop/V4/extracted_features_standardized.csv', sep=',')

if 'Unnamed: 0' in extracted_features_standardized.columns:
    extracted_features_standardized.drop(columns=['Unnamed: 0'], inplace=True)

display(extracted_features_standardized)

,AU01_r__sum_values,AU01_r__variance,AU01_r__standard_deviation,AU01_r__maximum,AU01_r__mean,AU01_r__root_mean_square,AU02_r__sum_values,AU02_r__variance,AU02_r__standard_deviation,AU02_r__maximum,...,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square,ID
0,-0.831420,-0.293678,-0.056030,0.514376,-0.487054,-0.246817,-0.774358,-0.309942,-0.164329,-0.013710,...,0.458331,-0.574983,-0.554028,-0.333560,0.221977,0.345618,0.548923,0.603537,0.429001,23
1,0.145470,-0.226965,0.005772,0.514376,-0.452632,-0.185153,0.433448,0.148409,0.398905,0.898596,...,0.458331,2.294469,2.317495,0.203487,-0.260024,-0.030426,0.619766,-0.659902,-0.281599,24
2,-0.394394,0.206189,0.372988,0.177477,-0.023172,0.245411,-0.796194,-0.485337,-0.431191,-0.339028,...,0.458331,0.196854,0.400873,-0.376228,0.033873,0.204639,-0.135892,0.093018,0.147314,25
3,0.215771,0.988392,0.933333,0.320838,0.644587,0.908811,-0.607925,-0.367648,-0.247737,-0.339028,...,0.458331,-0.847716,-0.761866,-0.868821,-0.323805,-0.084215,0.541052,-1.092069,-0.441394,26
4,2.729605,2.710656,1.914401,1.224014,3.093433,2.534479,5.079650,5.479450,3.809395,1.329997,...,1.863336,-2.230621,0.556978,2.456784,1.071168,0.914462,0.651252,3.692845,1.996765,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,1.859409,2.194404,1.645755,0.514376,1.258001,1.680854,1.440319,0.934144,1.140301,0.891524,...,0.458331,-0.446575,-0.318795,0.861706,0.199480,0.329100,0.210451,0.318307,0.327148,140
100,0.668573,0.605149,0.671849,0.514376,0.439073,0.631581,0.267418,0.124587,0.372873,0.898596,...,0.458331,-1.199405,-1.246095,-0.067533,-0.585183,-0.317237,-0.537335,-0.522513,-0.509273,142
101,-0.345492,-0.190675,0.038715,0.492872,-0.244453,-0.097095,-0.094241,0.168584,0.420737,0.573278,...,0.458331,-0.851589,-0.878346,-0.720364,-0.806836,-0.534298,-0.340549,-0.961821,-0.835709,144
102,0.580805,-0.031575,0.178060,0.514376,-0.358016,-0.013487,1.330607,0.251797,0.508808,0.849091,...,0.366881,-0.548002,-0.665071,1.190821,0.027513,0.199753,0.076637,-0.006872,0.113717,145


# Merging files 

In [78]:
# Assuming 'ID' is the common column in both DataFrames
standardized_complete_dataset = pd.merge(VVR_measurements_clean, extracted_features_standardized, on='ID', how='inner')

# Check the length of the merged DataFrame
print(len(standardized_complete_dataset))
display(standardized_complete_dataset)

104


,ID,Sum_12,Sum_4567,VVR_1,VVR_2,Sum_456,VVR_group,Condition,Date,Gender,...,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square
0,23,24.0,37.0,13.0,11.0,27.0,0,2,2020-08-01,2,...,-0.289518,0.458331,-0.574983,-0.554028,-0.333560,0.221977,0.345618,0.548923,0.603537,0.429001
1,24,23.0,37.0,12.0,11.0,28.0,0,2,2020-01-22,2,...,1.793425,0.458331,2.294469,2.317495,0.203487,-0.260024,-0.030426,0.619766,-0.659902,-0.281599
2,25,28.0,44.0,16.0,12.0,33.0,1,2,2020-05-02,2,...,0.557817,0.458331,0.196854,0.400873,-0.376228,0.033873,0.204639,-0.135892,0.093018,0.147314
3,26,30.0,37.0,15.0,15.0,29.0,0,1,2020-06-02,1,...,-0.394321,0.458331,-0.847716,-0.761866,-0.868821,-0.323805,-0.084215,0.541052,-1.092069,-0.441394
4,27,22.0,39.0,11.0,11.0,31.0,0,2,2020-06-02,1,...,1.682971,1.863336,-2.230621,0.556978,2.456784,1.071168,0.914462,0.651252,3.692845,1.996765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,140,16.0,32.0,8.0,8.0,24.0,0,3,2021-05-26,2,...,-0.033310,0.458331,-0.446575,-0.318795,0.861706,0.199480,0.329100,0.210451,0.318307,0.327148
100,142,20.0,34.0,11.0,9.0,26.0,0,3,2021-05-31,1,...,-0.853016,0.458331,-1.199405,-1.246095,-0.067533,-0.585183,-0.317237,-0.537335,-0.522513,-0.509273
101,144,24.0,35.0,12.0,12.0,27.0,0,3,2021-01-06,1,...,-0.565965,0.458331,-0.851589,-0.878346,-0.720364,-0.806836,-0.534298,-0.340549,-0.961821,-0.835709
102,145,20.0,37.0,11.0,9.0,28.0,0,1,2021-02-06,2,...,-0.485857,0.366881,-0.548002,-0.665071,1.190821,0.027513,0.199753,0.076637,-0.006872,0.113717


In [79]:
standardized_complete_dataset.to_csv('/Users/dionnespaltman/Desktop/V4/standardized_complete_dataset.csv', index=True)

In [80]:
# Count the number of NaN values in each column of the DataFrame
nan_counts = standardized_complete_dataset.isna().sum()

# Print columns with NaN values
nan_columns = nan_counts[nan_counts > 0]
print("Columns with NaN values:")
print(nan_columns)

Columns with NaN values:
Series([], dtype: int64)


# Get list of action units

In [36]:
import json

# Open the JSON file and load its contents into a list
with open('/Users/dionnespaltman/Desktop/V4/columns_action_units.json', 'r') as f:
    columns_action_units = json.load(f)

print("Loaded columns_action_units from JSON file:", len(columns_action_units))


Loaded columns_action_units from JSON file: 350


# Descriptives 

In [81]:
complete_dataset = pd.read_csv('/Users/dionnespaltman/Desktop/V4/standardized_complete_dataset.csv', sep=',')

if 'Unnamed: 0' in complete_dataset.columns:
    complete_dataset.drop(columns=['Unnamed: 0'], inplace=True)

display(complete_dataset)

,ID,Sum_12,Sum_4567,VVR_1,VVR_2,Sum_456,VVR_group,Condition,Date,Gender,...,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square
0,23,24.0,37.0,13.0,11.0,27.0,0,2,2020-08-01,2,...,-0.289518,0.458331,-0.574983,-0.554028,-0.333560,0.221977,0.345618,0.548923,0.603537,0.429001
1,24,23.0,37.0,12.0,11.0,28.0,0,2,2020-01-22,2,...,1.793425,0.458331,2.294469,2.317495,0.203487,-0.260024,-0.030426,0.619766,-0.659902,-0.281599
2,25,28.0,44.0,16.0,12.0,33.0,1,2,2020-05-02,2,...,0.557817,0.458331,0.196854,0.400873,-0.376228,0.033873,0.204639,-0.135892,0.093018,0.147314
3,26,30.0,37.0,15.0,15.0,29.0,0,1,2020-06-02,1,...,-0.394321,0.458331,-0.847716,-0.761866,-0.868821,-0.323805,-0.084215,0.541052,-1.092069,-0.441394
4,27,22.0,39.0,11.0,11.0,31.0,0,2,2020-06-02,1,...,1.682971,1.863336,-2.230621,0.556978,2.456784,1.071168,0.914462,0.651252,3.692845,1.996765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,140,16.0,32.0,8.0,8.0,24.0,0,3,2021-05-26,2,...,-0.033310,0.458331,-0.446575,-0.318795,0.861706,0.199480,0.329100,0.210451,0.318307,0.327148
100,142,20.0,34.0,11.0,9.0,26.0,0,3,2021-05-31,1,...,-0.853016,0.458331,-1.199405,-1.246095,-0.067533,-0.585183,-0.317237,-0.537335,-0.522513,-0.509273
101,144,24.0,35.0,12.0,12.0,27.0,0,3,2021-01-06,1,...,-0.565965,0.458331,-0.851589,-0.878346,-0.720364,-0.806836,-0.534298,-0.340549,-0.961821,-0.835709
102,145,20.0,37.0,11.0,9.0,28.0,0,1,2021-02-06,2,...,-0.485857,0.366881,-0.548002,-0.665071,1.190821,0.027513,0.199753,0.076637,-0.006872,0.113717


# Get the range of dates 

In [82]:
# Find the first and last dates
first_date = complete_dataset['Date'].min()
last_date = complete_dataset['Date'].max()

print("First date:", first_date)
print("Last date:", last_date)

First date: 2020-01-22
Last date: 2021-12-05


# Get the counts of condition

In [83]:
# Count of unique values in the 'Condition' column in clean_donor_info
complete_dataset_condition_count = complete_dataset['Condition'].value_counts()

# Display the counts
print("Count of Condition in the complete dataset")
print(complete_dataset_condition_count)

Count of Condition in the complete dataset
1    56
3    30
2    18
Name: Condition, dtype: int64


# Get the counts of VVR group

In [84]:
complete_dataset_VVR_count = complete_dataset['VVR_group'].value_counts()

# Display the counts
print("Count of VVR_group in the complete dataset")
print(complete_dataset_VVR_count)

Count of VVR_group in the complete dataset
0    81
1    23
Name: VVR_group, dtype: int64


# Distribution of gender 

In [85]:
complete_dataset_gender_count = complete_dataset['Gender'].value_counts()

# Display the counts
print("Count of gender in the complete dataset")
print(complete_dataset_gender_count)

Count of gender in the complete dataset
2    65
1    39
Name: Gender, dtype: int64


# Reducing features with TS Fresh using a relevance table

https://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_selection.html

In [88]:
# Check for missing values in any columns
missing_values_per_column = standardized_complete_dataset.isnull().any()

# Get the columns with missing values
columns_with_missing_values = missing_values_per_column[missing_values_per_column].index.tolist()

print("Columns with missing values:", columns_with_missing_values)

Columns with missing values: []


In [90]:
display(standardized_complete_dataset.head())

,ID,Sum_12,Sum_4567,VVR_1,VVR_2,Sum_456,VVR_group,Condition,Date,Gender,...,AU26_r__standard_deviation,AU26_r__maximum,AU26_r__mean,AU26_r__root_mean_square,AU45_r__sum_values,AU45_r__variance,AU45_r__standard_deviation,AU45_r__maximum,AU45_r__mean,AU45_r__root_mean_square
0,23,24.0,37.0,13.0,11.0,27.0,0,2,2020-08-01,2,...,-0.289518,0.458331,-0.574983,-0.554028,-0.333560,0.221977,0.345618,0.548923,0.603537,0.429001
1,24,23.0,37.0,12.0,11.0,28.0,0,2,2020-01-22,2,...,1.793425,0.458331,2.294469,2.317495,0.203487,-0.260024,-0.030426,0.619766,-0.659902,-0.281599
2,25,28.0,44.0,16.0,12.0,33.0,1,2,2020-05-02,2,...,0.557817,0.458331,0.196854,0.400873,-0.376228,0.033873,0.204639,-0.135892,0.093018,0.147314
3,26,30.0,37.0,15.0,15.0,29.0,0,1,2020-06-02,1,...,-0.394321,0.458331,-0.847716,-0.761866,-0.868821,-0.323805,-0.084215,0.541052,-1.092069,-0.441394
4,27,22.0,39.0,11.0,11.0,31.0,0,2,2020-06-02,1,...,1.682971,1.863336,-2.230621,0.556978,2.456784,1.071168,0.914462,0.651252,3.692845,1.996765


No features are seen as relevant. That could have something to do with the fact that we have a lot of features and very little data. 


In [91]:
from tsfresh.feature_selection.relevance import calculate_relevance_table

columns_to_drop = ['ID', 'Sum_12', 'Sum_4567', 'VVR_1', 'VVR_2', 'Sum_456', 'VVR_group', 'Condition', 'Date', 'Gender']

X = standardized_complete_dataset.drop(columns=columns_to_drop)
y = standardized_complete_dataset['VVR_group']

# Calculate relevance table
relevance_table = calculate_relevance_table(X, y, ml_task='classification', multiclass=False)

print("Relevance table:")
display(relevance_table)


Relevance table:


,feature,type,p_value,relevant
feature,,,,
AU02_r__maximum,AU02_r__maximum,real,0.040641,False
AU26_r__sum_values,AU26_r__sum_values,real,0.058037,False
AU26_r__mean,AU26_r__mean,real,0.068590,False
AU25_r__maximum,AU25_r__maximum,real,0.071150,False
AU09_r__sum_values,AU09_r__sum_values,real,0.083463,False
...,...,...,...,...
AU06_r__sum_values,AU06_r__sum_values,real,0.956277,False
AU02_r__sum_values,AU02_r__sum_values,real,0.956277,False
AU05_r__maximum,AU05_r__maximum,real,0.987500,False


In [92]:
# Count the number of relevant features
relevant_count = relevance_table['relevant'].value_counts()

print("Count of relevant features:")
print(relevant_count)


Count of relevant features:
False    102
Name: relevant, dtype: int64


# Reducing features with TS Fresh - select features 

In [93]:
columns_to_drop = ['Sum_12', 'Sum_4567', 'VVR_1', 'VVR_2', 'Sum_456', 'Condition', 'Date', 'Gender']

X = standardized_complete_dataset.drop(columns=columns_to_drop)
y = standardized_complete_dataset['VVR_group']

Double check if my input is correct. 

In [94]:
import pandas as pd

# Assuming X and y are already defined

# Check if X is a Pandas DataFrame
if isinstance(X, pd.DataFrame):
    print("X is a Pandas DataFrame")
else:
    print("X is not a Pandas DataFrame")

# Check if y is a Pandas Series
if isinstance(y, pd.Series):
    print("y is a Pandas Series")
else:
    print("y is not a Pandas Series")


X is a Pandas DataFrame
y is a Pandas Series


In [ ]:
from tsfresh import extract_features, select_features

# Remove double underscores from column names
X.columns = X.columns.str.replace('__', '_')

X_extracted = extract_features(X, column_id='VVR_group', column_sort='ID')
X_selected = select_features(X_extracted, y, ml_task='classification', multiclass=False)